## Youtube 댓글 Crawling

코랩에서 크롬드라이브 임포트

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 954 kB 5.3 MB/s 
     |████████████████████████████████| 138 kB 48.1 MB/s 
     |████████████████████████████████| 356 kB 46.9 MB/s 
     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 3.6 MB 19.4 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/

구글 드라이브 연동

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
from selenium import webdriver as wd
from datetime import datetime
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re
from pathlib import Path

# 오늘 날짜 폴더를 생성
today_date = datetime.today().strftime("%Y-%m-%d")
Path("/content/drive/MyDrive/Colab Notebooks/{}".format(today_date)).mkdir(parents=True, exist_ok=True)

# 유튜브 메인에서 인기기 페이지에 들어가 상위 10개의 영상에 대한 타이틀과 rul 수집(오늘 날짜 폴더에 titleCheck.txt로 따로 저장)
def get_urls_from_youtube():

    today_date = datetime.today().strftime("%Y-%m-%d")

    titles = []
    urls = []
    
    url = "https://www.youtube.com/feed/trending/?gl=KR"
    chrome_options = wd.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = wd.Chrome(executable_path="chromedriver", chrome_options=chrome_options)
    
    driver.get(url)
    
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        
        time.sleep(3.0)
        
        
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        if new_page_height == last_page_height:
            break
            
        last_page_height = new_page_height
        
    html_source = driver.page_source
    
    driver.quit()
    
    soup = BeautifulSoup(html_source, 'lxml')
    
    datas = soup.select("a#video-title")
    

    
    file_name = "/content/drive/MyDrive/Colab Notebooks/{}/titleCheck.txt".format(today_date)
    file_titles = open(file_name,"r")
    titleNm = file_titles.read()
    text = titleNm
    sendTitles = []
    sendUrls = []
    count = 10

    for data in datas:
        title = data.text.replace('\n', '')
        url = "https://www.youtube.com/" + data.get('href')
        if title in titleNm:
            continue
        else:
            count -= 1
            text += title + "\n"
            sendTitles.append(title)
            sendUrls.append(url)
        if count <= 0:
            break
    file_titles.close()
      
    file_titles = open(file_name,"w", encoding="utf-8")
    file_titles.write(text)
    file_titles.close()

    for i in sendTitles:
        print(i)

    for i in sendUrls:
        print(i)

    return sendTitles, sendUrls

# html 소스를 크롤링
def crawl_youtube_page_html_sources(urls):
    html_sources = []

    for i in range(len(urls)):
        chrome_options = wd.ChromeOptions()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')

        driver = wd.Chrome(executable_path="chromedriver", chrome_options=chrome_options)
        driver.get(urls[i])

        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(5.0)
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

            if new_page_height == last_page_height:
                break
            last_page_height = new_page_height


        html_source = driver.page_source
        html_sources.append(html_source)
        print("OK")

        driver.quit()
    return html_sources

# 크롤링한 html 소스에서 댓글 정보를 수집(타임라인 제거)
def get_comments(html_sources):
    my_dataframes = []
    for html in html_sources:
        
        soup = BeautifulSoup(html, 'lxml')

        youtube_comments = soup.select('yt-formatted-string#content-text')
        
        str_youtube_comments = []

        for i in range(len(youtube_comments)):
            str_tmp = str(youtube_comments[i].text)
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('               ', '')
            str_tmp = re.sub('\d{1,2}:\d{2}', '', str_tmp)
            str_youtube_comments.append(str_tmp)
            
        pd_data = {"Comment":str_youtube_comments}

        youtube_pd = pd.DataFrame(pd_data)

        my_dataframes.append(youtube_pd)
        
    return my_dataframes

# 오늘 날짜 폴더에 수집된 댓글들을 각 영상의 타이틀로 엑셀파일로 변환하여 저장
def convert_excel_from_dataframe(titles, my_dataframes):
  today_date = datetime.today().strftime("%Y-%m-%d")
  for i in range(len(my_dataframes)):  
        title = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…《\》]', '', titles[i])
        my_dataframes[i].to_excel("/content/drive/MyDrive/Colab Notebooks/{}/{}.xlsx".format(today_date, title)) 

## 사용 예시

In [5]:
titles, urls = get_urls_from_youtube()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: use options instead of chrome_options


오히려 좋아
Limousine (Feat. MINO) (Prod. by GRAY)
Meeting is easy, parting is hard (Feat. Leellamarz) (Prod. by TOIL)
Reguilon scores first ever Spurs goal | HIGHLIGHTS | Spurs 2-1 Leeds
SMTM10 [풀버전/8회] ♬ 리무진 (Feat. MINO) - 비오 @본선
Shall we dance?💥 The top-secret new member reveal💃of LACHICA🕺 | Dancer | LACHICA | WORKMAN ep.126
국밥집 공기밥의 밥알 개수는 몇 개일까?? 끝이 없다ㅋㅋㅋ
천만원빵 짜장면 5그릇 빨리먹기 푸드파이트.. 과연 승자는 누구??
빠지는 순간 얼음물 샤워! 공포의 징검다리 건너기 ㅋㅋㅋ 허징어게임#4
gganbu (Feat. YUMDDA & ASH ISLAND) (Prod. by TOIL)
https://www.youtube.com//watch?v=QcSskAs1F2s
https://www.youtube.com//watch?v=3eKFF4cCBaY
https://www.youtube.com//watch?v=BxfGh2xy7a4
https://www.youtube.com//watch?v=K05XHEB5YpU
https://www.youtube.com//watch?v=BRtDS6NDLJw
https://www.youtube.com//watch?v=4-dCVUL7zoM
https://www.youtube.com//watch?v=EdCQwCQ1xCM
https://www.youtube.com//watch?v=B6__3CLc-cg
https://www.youtube.com//watch?v=Xwtv06DsrvQ
https://www.youtube.com//watch?v=3oXKbxq1ceM


In [6]:
html_sorces = crawl_youtube_page_html_sources(urls)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: DeprecationWarning: use options instead of chrome_options


OK


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: DeprecationWarning: use options instead of chrome_options


OK


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: DeprecationWarning: use options instead of chrome_options


...

OK


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: DeprecationWarning: use options instead of chrome_options


OK


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: DeprecationWarning: use options instead of chrome_options


OK


In [14]:
my_dataframes = get_comments(html_sorces)

In [15]:
convert_excel_from_dataframe(titles, my_dataframes)

최종적으로 유튜브 메인에서 인기 페이지의 상위 10개의 영상에 대한 댓글들을 수집하여 오늘 날짜 폴더에 각 영상의 타이틀명으로 엑셀 파일 형태로 저장